In [53]:
import math
from math import comb
import itertools
import argparse

In [54]:
k = 6
l = 2
r = 2
Locality = 3
n = 10
#cross rack repair bandwidth in Gbps
B = 1
#disk capcity in TB
S = 16
N = 400
#Repair cost
Rc = (3*8 + 6*2)/10
Pd = .857
#In years
MTTF = 4
#failure rate
f = (1/MTTF)
f_h = f/8760
print(f_h)
#epsilon
ep = 0.1
#error detection time in sec
T = 1800


2.8538812785388127e-05


In [55]:
rho1 = (ep*(N-1)*B)/(8*S*1000*Rc)
rho2 = 1/T
rho3 = rho2
rho4 = rho2
rho1_h = rho1*3600
rho2_h = rho2*3600
rho3_h = rho2_h
rho4_h = rho2_h
print("rho1 = ", rho1_h)
print("rho2 = ", rho2_h)


rho1_by_lambda = rho1_h/f_h
rho2_by_lambda = rho2_h/f_h

print("rho1_by_lambda", rho1_by_lambda)
print("rho2_by_lambda", rho2_by_lambda)


MTTDLstripe = (rho1_by_lambda*rho2_by_lambda**2)/(10*9*8*7*f*(1-Pd))
print("MTTDLstripe = ", MTTDLstripe)


print("log of MTTDLstripe", math.log10(MTTDLstripe))

rho1 =  0.31171875000000004
rho2 =  2.0
rho1_by_lambda 10922.625000000002
rho2_by_lambda 70080.0
MTTDLstripe =  297720422937.063
log of MTTDLstripe 11.473808627272348


In [56]:
def calculate_recoverability(k, l, r, R):
    n = k + l + r

    data_blocks = []
    for i in range(k):
        data_blocks.append('k'+str(i))
    print(data_blocks)


    local_parities = []
    for i in range(l):
        local_parities.append('l'+str(i))
    print(local_parities)


    global_parities = []
    for i in range(r):
        global_parities.append('r'+str(i))
    print(global_parities)

    data_and_parity = data_blocks + local_parities + global_parities
    print(data_and_parity)
    
    all_comb = list(itertools.combinations(data_and_parity, R))
    
    num_local_groups = l
    local_groups = []

    local_group_ids = []
    
    start = 0
    end = k
    step = int(k/l)
    for i in range(start, end, step):
        x = i
        local_groups.append(data_blocks[x:x+step])
    
    print(local_groups)
    recoverable = 0
    for i in range(len(all_comb)):
        k_terms = 0
        l_terms = l
        r_terms = r
        comb_list = list(all_comb[i])
        #print(comb_list)
        for j in range(len(all_comb[i])):
            if 'k' in comb_list[j]:
                k_terms = k_terms + 1
        #print("   ", k_terms, l_terms, r_terms)
        for j in range(len(all_comb[i])):
            if 'k' in all_comb[i][j]:
                for m in range(len(local_groups)):
                    if all_comb[i][j] in local_groups[m]:
                        local_group_id = m
                        break
                #print("   local_group_id = ", local_group_id)
                if local_parities[local_group_id] not in comb_list:
                    k_terms = k_terms - 1
                    comb_list.append(local_parities[local_group_id])
            if 'r' in all_comb[i][j]:
                r_terms = r_terms - 1
        #print("   ", k_terms, r_terms)
        if k_terms <= r_terms:
            recoverable = recoverable + 1
            #print(list(all_comb[i]), "-R")
        #else:
            #print(list(all_comb[i]), "-N")
        #print("   recoverable = ", recoverable)
        
    return (recoverable/len(all_comb))

In [59]:
def calculate_MTTDLstripe(k, l, r, ep, N, B, S, f, T):

    n = k+l+r
    Rc = ((k/l)*(k+l) + k*r)/n

    f_h = f/8760

    rho1 = (ep*(N-1)*B)/(8*S*1000*Rc)
    rho2 = 1/T
    rho1_h = rho1*3600
    rho2_h = rho2*3600

    rho1_by_lambda = rho1_h/f_h
    rho2_by_lambda = rho2_h/f_h

    part_failure = r+2
    Pd = calculate_recoverability(k,l,r, part_failure)
    print("Pd = ", Pd*100)
    #Pd = 0.9999
    num = rho1_by_lambda*rho2_by_lambda**(part_failure-2)
    denom = 1
    for i in range(part_failure):
        denom = denom*(n-i)

    denom = denom*f*(1-Pd)

    MTTDLstripe = num/denom
    
    return MTTDLstripe, math.log10(MTTDLstripe)

In [64]:
MTTDLstripe, logMTTDL = calculate_MTTDLstripe(42, 6, 12, 0.1, 400, 1, 16, 0.25, 1800)
print("MTTDLstripe = ", MTTDLstripe)
print("log of MTTDLstripe", logMTTDL)

MTTDLstripe =  1.0425225107336717e+42
log of MTTDLstripe 42.01808544127828
